In [ ]:
import black
import jupyter_black

jupyter_black.load(
    line_length=79,
    verbosity="DEBUG",
    target_version=black.TargetVersion.PY314,
)

# Solution to Exercise set 5

The main goals of this exercise are to use PCA

**Learning Objectives:**

After completing this exercise set, you will be able to:

- Use PCA to find interesting variables.
- Find outliers in connection with least squares.
- Run agglomerative clustering and interpret a dendrogram.

**To get the exercise approved, complete the following problems:**

* [5.2(b)](#5.2(b)) and [5.2(c)](#5.2(c)): To show that you find outliers in connection with least squares.
* [5.3(a)](#5.3(a)) and [5.3(b)](#5.3(b)): To show that you can run agglomerative clustering, make a dendrogram and interpret it.

**Files required for this exercise:**
* [Exercise 5.1](#Exercise-5.1): [ovo.csv](ovo.csv)
* [Exercise 5.2](#Exercise-5.2): [forbes.csv](forbes.csv)
* [Exercise 5.3](#Exercise-5.3): [zoo.csv](zoo.csv)

Please ensure that these files are saved in the same directory as this notebook.

## Exercise 5.1
[Schummer *et al.*](https://doi.org/10.1016/S0378-1119(99)00342-X) used microarray technology to analyse the expression of 1536 genes in ovarian cancer and non-cancer tissues. Their primary objective was to identify differentially expressed genes in ovarian cancer versus non-cancer tissues to discover genes with diagnostic potential.

The data file [`ovo.csv`](ovo.csv) contains numerical gene expressions (for 1536 genes) for 54 tissue samples. Each column corresponds to a specific gene, named `X.1`, `X.2`, and so on. Each tissue sample has been classified as non-cancer (`N`) or cancer (`C`) tissue, and these labels can be found in the column `class`. The raw data has been preprocessed by centring each gene expression so that no further preprocessing is needed. The raw data can be loaded as follows:

In [ ]:
# Load the data set
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline
sns.set_theme(style="ticks", context="notebook", palette="colorblind")

data_ovo = pd.read_csv("ovo.csv")
classes = data_ovo["class"]  # Classification of samples.
X_ovo = data_ovo.filter(like="X.", axis=1)  # Gene expressions for samples.

### 5.1(a)

**Task: Explore the raw data. Do you find genes that appear to show significant differences in expression between non-cancer and cancer tissue?**

**Hint:** You can, for instance, explore the data by running a principal component analysis. If you want to avoid making the PCA plots from scratch, try the [yellowbrick library](https://www.scikit-yb.org/en/latest/api/features/pca.html) to visualise the scores and loadings.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
scores = pca.fit_transform(X_ovo)

In [ ]:
fig, axes = plt.subplots(constrained_layout=True, ncols=2, figsize=(8, 4))

sns.scatterplot(
    data=data_ovo, x=scores[:, 0], y=scores[:, 1], hue="class", ax=axes[0]
)

sns.scatterplot(
    data=data_ovo, x=scores[:, 0], y=scores[:, 2], hue="class", ax=axes[1]
)
percent = pca.explained_variance_ratio_ * 100

axes[0].set(
    xlabel=f"Scores, PC1 ({percent[0]:.2g}%)",
    ylabel=f"Scores, PC2 ({percent[1]:.2g}%)",
)


axes[1].set(
    xlabel=f"Scores, PC1 ({percent[0]:.2g}%)",
    ylabel=f"Scores, PC3 ({percent[2]:.2g}%)",
)


for ax in axes:
    ax.axhline(y=0, ls=":", color="k")
    ax.axvline(x=0, ls=":", color="k")

sns.despine(fig=fig)

The PC1 scores separate the two tissue types. To understand what variables are important for this separation, we inspect the loadings:

In [ ]:
import matplotlib.patheffects as pe


loadings = pca.components_.T
pc1_loadings = loadings[:, 0]
pc2_loadings = loadings[:, 1]
percent = pca.explained_variance_ratio_ * 100


fig, ax = plt.subplots(constrained_layout=True)

ax.set(
    xlabel=f"Loadings, PC1 ({percent[0]:.2g}%)",
    ylabel=f"Loadings, PC2 ({percent[1]:.2g}%)",
)
ax.axhline(y=0, ls=":", color="k")
ax.axvline(x=0, ls=":", color="k")

# Get the 10 largest loadings along PC1 for highlighting:
highlighted_indices = np.argsort(abs(pc1_loadings))[-10:]
print(f"10 largest along PC1: {highlighted_indices}")

for i, (xi, yi) in enumerate(zip(pc1_loadings, pc2_loadings)):
    if i in highlighted_indices:
        txt = ax.text(xi, yi, i, fontsize="small", ha="center", va="center")
        txt.set_path_effects(
            [
                pe.withStroke(linewidth=1.5, foreground="yellow"),
                pe.Normal(),
            ]
        )
    else:
        txt = ax.text(
            xi, yi, i, fontsize="small", ha="center", va="center", color="0.7"
        )


ax.set_xlim(-0.15, 0.15)
ax.set_ylim(-0.15, 0.15)
sns.despine(fig=fig)

Let us check two of the highlighted genes by creating a scatterplot of the original samples:

In [ ]:
gene1 = 1490
gene2 = 92
grid = sns.jointplot(
    data=data_ovo,
    x=f"X.{gene1 + 1}",
    y=f"X.{gene2 + 1}",
    hue="class",
)
ax = grid.fig.axes[0]
ax.set_xlabel(f"Gene expression for X.{gene1 + 1}")
ax.set_ylabel(f"Gene expression for X.{gene2 + 1}")

#### Your answer to question 5.1(a): Did you find any promising genes?

Yes, genes with the highest absolute loadings on Principal Component 1 (PC1) appear promising for distinguishing between the samples. For example, number (counting from zero) 1490 and number 92 as shown above.

## Exercise 5.2
[Forbes](https://doi.org/10.1017/S0080456800032075) investigated the
relationship between the boiling point of water and the atmospheric pressure, and collected data in the Alps and Scotland. Forbes' goal was to estimate altitudes from the boiling point alone.

We will use Forbes' data to make a linear model for predicting the atmospheric pressure, and we will investigate if there are any outliers.

### 5.2(a)

**Task: Create a linear model that predicts the atmospheric pressure
from the boiling point
with `statsmodels`. Plot your model together with the raw data, and plot the residuals. Do you have
any comments about the residuals?**

In [ ]:
# Here is some code to get you started:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import pandas as pd

%matplotlib inline
sns.set_context("notebook")

In [ ]:
# If the code above fails because you do not have statsmodels installed,
# you can uncomment and run this:
#!pip install statsmodels

In [ ]:
# The raw data can be loaded with:
data_forbes = pd.read_csv("forbes.csv")
data_forbes.head()

In [ ]:
# To create the least squares model with statsmodels:
x = data_forbes["Temperature (F)"]
y = data_forbes["Pressure (inches Hg)"]

X = sm.add_constant(x)  # Make a matrix with a column of ones and then x.

model = sm.OLS(y, X)  # Set up for OLS = Ordinary Least Squares.
results = model.fit()  # Find parameters.
y_hat = results.predict(X)  # Use the model to predict y_hat.

# Print a small summary to show R² and the coefficients:
print(results.summary(slim=True))

**Note:** A description of the summary from statsmodels can be found in the [Appendix](#Appendix:-The-summary-results-from-statsmodels).

In [ ]:
fig, axes = plt.subplots(constrained_layout=True, ncols=2, figsize=(8, 4))

axes[0].scatter(x, y, label="Raw data")
axes[0].set(xlabel="Temperature (F)", ylabel="Pressure (inches Hg)")
axes[0].plot(
    x,
    y_hat,
    label=f'Model: y = {results.params["const"]:.3g} + {results.params["Temperature (F)"]:.3g}x',
)
axes[0].legend()

axes[1].scatter(y_hat, y - y_hat)
axes[1].set(xlabel="ŷ", ylabel="y-ŷ")
axes[1].axhline(y=0, ls=":", color="k")
sns.despine(fig=fig)

#### Your answer to question 5.2(a): Do you have any comments about the residuals?

There seems to be some kind of trend in the residuals, and one of the points is far away from the others.
It could be that this is an outlier.

### 5.2(b)

**Task: Obtain different outlier measures and plot them. Do you see any potential outliers?**

**Hint**: Calculate [influence/outlier measures with statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.OLSInfluence.html#statsmodels.stats.outliers_influence.OLSInfluence) and plot the following:

* (i) the studentised residuals,
* (ii) the leverage ($h_{ii}$ from the $\mathbf{H}$-matrix),
* (iii) the Cook's distance, and
* (iv) the [influence plot](https://www.statsmodels.org/dev/generated/statsmodels.graphics.regressionplots.influence_plot.html).

You can find example code for doing this below.

In [ ]:
# The influence measures can be computed with:
influence = results.get_influence()

# The measures can be converted to a pandas data frame with:
influence_table = influence.summary_frame()

# And they can be accessed as follows:
# (i) studentised residuals:
studentised_residuals = influence_table["student_resid"]

# (ii) the leverage:
hii = influence_table["hat_diag"]

# (iii) Cook's distance
cooks_distance = influence_table["cooks_d"]

In [ ]:
fig, axes = plt.subplots(constrained_layout=True, ncols=3, figsize=(9, 3))

axes[0].scatter(y_hat, studentised_residuals)
axes[0].axhline(y=-2, ls=":", color="k")
axes[0].axhline(y=2, ls=":", color="k")
axes[0].set(xlabel="ŷ", ylabel="Studentised residuals")
axes[0].set_title("Residuals", loc="left")
axes[0].set_ylim(-4.5, 4.5)

xpos = np.arange(len(hii))
axes[1].bar(xpos, hii)
axes[1].set(xlabel="Observation no.", ylabel="$h_{ii}$")
axes[1].set_title("Influence", loc="left")

axes[2].bar(xpos, cooks_distance)
axes[2].set(xlabel="Observation no.", ylabel="$D_{i}$")
axes[2].set_title("Cook's distance", loc="left")
sns.despine(fig=fig)

In [ ]:
# The influence plot, part (iv) can be created with:
fig = influence.plot_influence()
ax = fig.get_axes()[0]
ax.set_ylim(-1.3, 3.5)
ax.set_title("")
sns.despine(fig=fig)

#### Your answer to question 5.2(b): Do you see any potential outliers?

Yes, one point is a bit suspicious: observation no. 11 (counting from zero) has a large studentised residual and a large influence (Cook's distance). This could be a potential outlier.

### 5.2(c)

**Task: Run a hypothesis test for outliers. Are any points marked as outliers?**

**Hint:** You use the [outlier test](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLSResults.outlier_test.html)
method from `statsmodels` to run the hypothesis test (see the code in the cell below).

This test outputs:
* `student_resid`: The studentised residuals.
* `unadj_p`: The unadjusted p-value for the hypothesis test that the expected value of the studentized residual for point *i* is zero, under the null hypothesis that the point is not an outlier.
* `bonf(p)`: A Bonferroni corrected p-value, which adjusts for the increased risk of Type I errors (mistaken rejection of a true null hypothesis) due to multiple comparisons.

The `outlier_test` method tests the null hypothesis that each point is not an outlier by considering if its studentized residual is significantly different from zero. Since we perform this test *N* times for *N* points, the risk of incorrectly labelling at least one point as an outlier (Type I error) increases. To mitigate this, `outlier_test` will apply a [correction](https://en.wikipedia.org/wiki/Bonferroni_correction), and we should base our decisions on these corrected p-values.

You can identify a point as an outlier if the corrected p-value is smaller than the significance level, set to `alpha = 0.05` by default in the `outlier_test()` method.

In [ ]:
# This is how you run the hypothesis test:
test = results.outlier_test()
test[test["bonf(p)"] < 0.05]

#### Your answer to question 5.2(c): Were any points identified as outliers?

Yes, point no. 11 is identified as an outlier!

### 5.2(d)

**Task: You should have found one outlier in the previous problem. Remake the model without this point and
compare it with the model you made in part [5.2(a)](#5.2(a)). Did the model change substantially?**

In [ ]:
# We remove the suspicious point:
x_new = x.drop(11)
y_new = y.drop(11)
X_new = sm.add_constant(x_new)

# And remake the model:
new_model = sm.OLS(y_new, X_new).fit()

y_hat_new = new_model.predict(X_new)

print(new_model.summary(slim=True))

In [ ]:
fig, axes = plt.subplots(constrained_layout=True, ncols=2, figsize=(8, 4))

axes[0].scatter(x, y, label="Raw data")
axes[0].set(xlabel="Temperature (F)", ylabel="Pressure (inches Hg)")
axes[0].plot(
    x,
    y_hat,
    label=f'Model 1: y = {results.params["const"]:.3g} + {results.params["Temperature (F)"]:.3g}x',
)
axes[0].plot(
    x_new,
    y_hat_new,
    label=f'Model 2: y = {new_model.params["const"]:.3g} + {new_model.params["Temperature (F)"]:.3g}x',
)
axes[0].legend()

axes[1].scatter(y_hat, y - y_hat, label="Model 1")
axes[1].scatter(y_hat_new, y_new - y_hat_new, label="Model 2")
axes[1].set(xlabel="ŷ", ylabel="y-ŷ")
axes[1].axhline(y=0, ls=":", color="k")
axes[1].legend()
sns.despine(fig=fig)

#### Your answer to question 5.2(d): Did removing the outlier change the model substantially?

Removing the point does not seem to change the model a lot. The calculated Cook's distance is not a lot
bigger for this point, compared to some of the other points. So in this case, the outlier was not so
important for the model.

## Exercise 5.3
The data file [zoo.csv](zoo.csv) contains some data on different animals:

| Column | Description |
| :---  | :--- |
| `animal name` | The name for the animal (e.g., lion, penguin). |
| `hair` | Does the animal have hair or fur? (0=no/1=yes) |
| `feathers` | Does the animal have feathers? (0=no/1=yes) |
| `eggs` | Does the animal lay eggs? (0=no/1=yes) |
| `milk` | Does the animal provide milk for its young? (0=no/1=yes) |
| `airborne` | Does the animal fly? (0=no/1=yes) |
| `aquatic` | Does the animal live in or depend on water? (0=no/1=yes) |
| `predator` | Does the animal hunt other animals? (0=no/1=yes) |
| `toothed` | Does the animal have teeth? (0=no/1=yes) |
| `backbone` | Does the animal have a spine? (0=no/1=yes) |
| `breathes` | Does it breathe air (using lungs)? (0=no/1=yes) |
| `venomous` | Does the animal produce toxins or venom? (0=no/1=yes) |
| `fins` | Does the animal have fins for swimming? (0=no/1=yes) |
| `legs` | The number of legs (normalised to the range 0 to 1). |
| `tail` | Does the animal have a tail? (0=no/1=yes) |
| `domestic` | Is it commonly domesticated by humans? (0=no/1=yes)|
| `catsize` | Is the animal larger than a housecat? (0=no/1=yes) |


We will use clustering on this data set to see if we can group animals.

### 5.3(a)

**Task: Run the code below to perform [Agglomerative clustering](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) for the data in [zoo.csv](zoo.csv).**


**Note:** As you may have noticed, we have many binary variables and one numerical variable (`legs`). Mixing variable types can be problematic when we are calculating a distance. We will ignore the potential issues related to this in this exercise. 

In [ ]:
# Import libraries:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib notebook
sns.set_context("notebook")

In [ ]:
# Read the data:
data_zoo = pd.read_csv("zoo.csv")
data_zoo.head()

In [ ]:
# Store some variables
animal_names = data_zoo["animal name"].values
variables = [i for i in data_zoo.columns if i not in ("animal name",)]
X_zoo = data_zoo[variables]

In [ ]:
# Import the clustering class:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
# Define the clustering:
model0 = AgglomerativeClustering(
    n_clusters=1,
    linkage="average",
    metric="sokalmichener",
    compute_distances=True,
)


# Here:
# n_clusters: Stop the clustering when n_clusters are found. Here it is set to 1, meaning that we will stop
# when there is one cluster.
# linkage: Selects the method for computing distances between clusters. Here it is set to average, meaning that
# it calculates the average distance between the points in the clusters.
# metric: Selects how the distances are calculated. Here "sokalmichener" selects a Sokal-Michener distance that takes
# both matches and dissimilarities into account. We are not using an Euclidean distance since we have
# almost exclusively boolean variables (0/1).
# compute_distances: Set to True, meaning that distances (for visualisation in a dendrogram) are computed.

# Run the clustering:
model0.fit(X_zoo)

**Note:** You can select different options for the `linkage` and the `metric` to see how this influences the results (the dendrogram produced in the next problem). Please see the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html) for options.

### 5.3(b)

**Task: Run the code below to create the dendrogram of the clustering performed above. Then consider the following:**
1. What animal is "human" most similar to?
2. What animal is "platypus" most similar to?
3. What animal is least similar to any of the others?

In [ ]:
# To create the dendrogram, use this code:
from scipy.cluster.hierarchy import dendrogram


def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram
    # Create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

In [ ]:
fig, axes = plt.subplots(constrained_layout=True, figsize=(8, 4))
plot_dendrogram(
    model0,
    truncate_mode="level",
    labels=animal_names,
    ax=axes,
    p=100,
    leaf_rotation=90,
)
sns.despine(fig=fig)

#### Your answer to question 5.3(b):

**Note:** Your answer to these might depend on the selected `linkage` and `metric`.

1. What animal is "human" most similar to? 
   - The human is merged first with the bear. In hierarchical clustering, the earliest
     merges represent the smallest distances, indicating that the human is more similar to
     the bear than to any other animal
2. What animal is "platypus" most similar to?
   - The platypus merges with a cluster of mammals (including the fruitbat and elephant).
     This indicates that despite being an egg-layer, it shares mammalian traits (like milk and hair)
     make it more similar to the mammals
3. What animal is least similar to any of the others?
   - The honeybee is the animal that is merged last with the other animals,
     indicating that it is different from the rest (there are no other insects in this data set).